# **Logistic Regression**
수학을 사용해서 두 데이터 요인간의 관계를 찾는 분석 기법.   
예측도 Y/N과 같은 유한한 수를 가짐.(이름은 Regression이지만 사실 Binary Classification 문제)  
</br>
**예시)**  
❔ 웹 쇼핑몰 방문자가 과연 결제 버튼을 누를까요?  
&nbsp;&nbsp;&nbsp; Input = 웹 서핑 시간 + 카트에 담은 항목 수 + ..등등   
&nbsp;&nbsp;&nbsp;Ouput = 누를까? 말까?   
</br>
### ⚡**장점**
1. 다른 ML 모델보다 간단한 수식
2. 대량의 data 고속 처리 가능
    - 메모리 처리 및 성능과 같은 메모리 계산 용량이 덜 필요
3. 전처리에도 사용 OK! 범위가 넓은 DATA를 작고 유한한 값 범위로 정렬
4. 계산이 덜 복잡해서 오류 수정도 더 쉽다

### ⚡**응용**
- 제조
    - 기계류의 부품이 고장날 확률 추정. 고장 나 안 나?
- 의료
    - 질병 발생 가능성 예측
- 금융
    - 사기 / NOT 사기
- 마케팅
    - 광고 클릭 / 클릭 안 함
</br></br>

### ⚡**활성화 함수: Logistic 함수**
$$ H(X) = \frac{1}{1+e^{-W^T X}} = \frac{1}{1+e^{-XW}}$$    
 - If $y \simeq H(x)$, cost is near 0.
 - If $y \neq H(x)$, cost is high.
<br>

H(x): 어떤 Sample이 1일 확률
</br></br>

Sigmoid 함수중 하나로, sigmoid 곡선을 가진 함수. Sigmoid와 마찬가지로 모든 점에서 음이 아닌 미분 값을 가짐.   
(1과 0 사이의 값을 가짐.)           
</br>

**➕) sigmoid 함수 종류**   
- tanh 함수
- logistic 함수   

등   

### ⚡**Cost**
$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$
<br>

### ⚡**Gradient Descent를 이용한 가중치 Update**
$$ W := W - \alpha \frac{\partial}{\partial W} cost(W) = W - \alpha ∇W$$
 - $\alpha$: Learning rate

## Imports

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#for reproducibility
torch.manual_seed(1) # Set the seed for generating random numbers. Return a torch.Generator object
                     # Seed: 동일한 세트의 난수. 약간 원인? 뿌리? 느낌으로 기억하자
                     # Random seed를 고정하기 위함! 안 그러면 값이 달라져요
                     # 1은 시드 번호

In [4]:
#강의를 듣는 시간, 코딩을 하는 시간이 주어졌을 때, 해당 코스를 통과하는지 낙제하는지를 나타내는 dataset.
x_data = [[1, 2],[2, 3],[3, 1],[4, 3],[5, 3],[6, 2]] #|x_data| = (6, 2)
y_data = [[0], [0], [0], [1], [1], [1]] #|y_data| = (6, 1)

In [5]:
#받아온 data를 torch로 바꿔준다.
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [6]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis
bias도 고려해 봅시다.
$$ H(X) =  \frac{1}{1+e^{-XW}}$$
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;⬇
$$ H(X) =  \frac{1}{1+e^{-(xw + b)}}$$

파이토치는 지수함수를 닮은 function인 torch.exp()를 지원합니다.

In [7]:
print('e^1 equeals: ', torch.exp(torch.FloatTensor([1])))

e^1 equeals:  tensor([2.7183])


 torch.exp()를 Hypothesis에 써 보아요

In [8]:
W = torch.zeros((2, 1), requires_grad = True)
b= torch.zeros(1, requires_grad=True)

#hypothesis = 1 / (1 + torch.exp(x_train.matmul(W) + b))
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


## Computing the Cost Function (Low-level)

In [10]:
# element 하나일 때(즉, 스칼라일 때)
# -(y_train[0] * torch.log(hypothesis[0]))+(1-y_train[0])*(torch.log(1-hypothesis[0]))

#vector 계산
losses = - (y_train * torch.log(hypothesis))+(1-y_train)*(torch.log(1-hypothesis))

print(losses)

tensor([[-0.6931],
        [-0.6931],
        [-0.6931],
        [ 0.6931],
        [ 0.6931],
        [ 0.6931]], grad_fn=<AddBackward0>)


In [12]:
cost = losses.mean()
print(cost)

tensor(0., grad_fn=<MeanBackward0>)


### Binary Cross Entropy Loss
위에서 적은 cost 구하는 공식이 바로바로 Binary Cross Entropy 입니당.
학습을 통해 얻은 확률 분포와, 예측 분포간의 차를 측정하는 함수.   
Cross Entropy와 달리 하나의 확률만 저장함.   
예로,   
동전 던지기에서 P(앞면) = 0.7로 측정되었다면, 뒷면이 나올 확률을 0.3으로 가정함.   
Cross Entropy는 앞면 뒷면 확률 다 저장!

In [13]:
#cost 구하는 코드를 아래와 같이 간단히 할 수 있다.
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

### Whole Training Procedure

In [41]:
x_data = [[1, 2],[2, 3],[3, 1],[4, 3],[5, 3],[6, 2]] #|x_data| = (6, 2)
y_data = [[0], [0], [0], [1], [1], [1]] #|y_data| = (6, 1)

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

W =  torch.zeros((2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W,b], lr=0.1)

nb_epoches = 1200

for epoch in range(1, nb_epoches+1):
    hypothesis = torch.sigmoid(x_train.matmul(W)+b)

    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} cost={:.6f}'.format(epoch, nb_epoches, cost.item()))
        print('H(x): ', hypothesis.squeeze(dim=1).detach())

Epoch  100/1200 cost=0.415135
H(x):  tensor([0.2175, 0.2645, 0.6875, 0.6292, 0.7866, 0.9307])
Epoch  200/1200 cost=0.350068
H(x):  tensor([0.1472, 0.2205, 0.6389, 0.6497, 0.8261, 0.9500])
Epoch  300/1200 cost=0.301727
H(x):  tensor([0.1128, 0.2077, 0.5762, 0.6734, 0.8525, 0.9566])
Epoch  400/1200 cost=0.263868
H(x):  tensor([0.0897, 0.2000, 0.5171, 0.6954, 0.8734, 0.9613])
Epoch  500/1200 cost=0.233787
H(x):  tensor([0.0725, 0.1928, 0.4655, 0.7149, 0.8904, 0.9654])
Epoch  600/1200 cost=0.209531
H(x):  tensor([0.0594, 0.1855, 0.4214, 0.7321, 0.9044, 0.9692])
Epoch  700/1200 cost=0.189676
H(x):  tensor([0.0493, 0.1782, 0.3839, 0.7472, 0.9161, 0.9726])
Epoch  800/1200 cost=0.173186
H(x):  tensor([0.0413, 0.1711, 0.3520, 0.7606, 0.9257, 0.9756])
Epoch  900/1200 cost=0.159307
H(x):  tensor([0.0350, 0.1642, 0.3245, 0.7725, 0.9339, 0.9783])
Epoch 1000/1200 cost=0.147484
H(x):  tensor([0.0299, 0.1577, 0.3008, 0.7832, 0.9407, 0.9806])
Epoch 1100/1200 cost=0.137302
H(x):  tensor([0.0258, 0.1515,

In [46]:
#소수로 나오는 hypothesis를 binary하게 0과 1로 다음과 같이 나타낼 수 있다
prediction = hypothesis.float() >= torch.FloatTensor([0.5]) #ByteTensor를 형변환
print(prediction[:5].type(torch.uint8))

tensor([[0],
        [0],
        [0],
        [1],
        [1]], dtype=torch.uint8)


In [48]:
#이제 y_train과 예측 값을 비교해 보자
print(prediction[:5].type(torch.uint8))
print(y_train[:5])

tensor([[0],
        [0],
        [0],
        [1],
        [1]], dtype=torch.uint8)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [51]:
#prediction과 값이 같으면 1을 출력.
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5].type(torch.uint8))

tensor([[1],
        [1],
        [1],
        [1],
        [1]], dtype=torch.uint8)


In [64]:
#모델을 class로 정의해봐요
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

x_data = [[1, 2],[2, 3],[3, 1],[4, 3],[5, 3],[6, 2]] #|x_data| = (6, 2)
y_data = [[0], [0], [0], [1], [1], [1]] #|y_data| = (6, 1)

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

model = BinaryClassifier()

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epoches = 1200
for epoch in range(1, nb_epoches+1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch: {:4}/{} Cost: {:.6f}, Accuracy: {:2.2f}%'.format(epoch, nb_epoches, cost.item(), accuracy * 100))

Epoch:  100/1200 Cost: 0.434244, Accuracy: 83.33%
Epoch:  200/1200 Cost: 0.366090, Accuracy: 83.33%
Epoch:  300/1200 Cost: 0.314276, Accuracy: 83.33%
Epoch:  400/1200 Cost: 0.273754, Accuracy: 83.33%
Epoch:  500/1200 Cost: 0.241682, Accuracy: 100.00%
Epoch:  600/1200 Cost: 0.215933, Accuracy: 100.00%
Epoch:  700/1200 Cost: 0.194945, Accuracy: 100.00%
Epoch:  800/1200 Cost: 0.177584, Accuracy: 100.00%
Epoch:  900/1200 Cost: 0.163026, Accuracy: 100.00%
Epoch: 1000/1200 Cost: 0.150666, Accuracy: 100.00%
Epoch: 1100/1200 Cost: 0.140053, Accuracy: 100.00%
Epoch: 1200/1200 Cost: 0.130848, Accuracy: 100.00%
